<a href="https://colab.research.google.com/github/stsibin/ML-projects/blob/master/LogReg_OTC_bank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from pandas import Series, DataFrame
from collections import Counter, defaultdict
from sklearn.impute import SimpleImputer
import custom_preprocessing as custprep
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

pd.set_option('display.max_columns', 60)
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Артем Груздев Курс/Course_ML/Модуль_3/Data/Credit_OTP.csv', encoding='cp1251', sep=';')
data.head()

,AGREEMENT_RK,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,EDUCATION,MARITAL_STATUS,GEN_INDUSTRY,GEN_TITLE,ORG_TP_STATE,ORG_TP_FCAPITAL,JOB_DIR,FAMILY_INCOME,PERSONAL_INCOME,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,TP_PROVINCE,REGION_NM,REG_FACT_FL,FACT_POST_FL,REG_POST_FL,REG_FACT_POST_FL,REG_FACT_POST_TP_FL,FL_PRESENCE_FL,OWN_AUTO,AUTO_RUS_FL,HS_PRESENCE_FL,COT_PRESENCE_FL,GAR_PRESENCE_FL,LAND_PRESENCE_FL,CREDIT,TERM,FST_PAYMENT,DL_DOCUMENT_FL,GPF_DOCUMENT_FL,FACT_LIVING_TERM,WORK_TIME,FACT_PHONE_FL,REG_PHONE_FL,GEN_PHONE_FL,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_NUM_PAYM,LOAN_DLQ_NUM,LOAN_MAX_DLQ,LOAN_AVG_DLQ_AMT,LOAN_MAX_DLQ_AMT,PREVIOUS_CARD_NUM_UTILIZED
0,59910150,0,49,1,0,1,2,1,Среднее специальное,Состою в браке,Торговля,Рабочий,Частная компания,Без участия,Вспомогательный техперсонал,от 10000 до 20000 руб.,5000,Оренбургская область,Оренбургская область,Оренбургская область,Оренбургская область,ПОВОЛЖСКИЙ,1,1,1,1,1,0,0,0,0,0,0,0,8000,6,8650,0,1,220,18.000,0,0,1,1,1,6,2,1,1580,1580,nan
1,59910230,0,32,1,0,1,3,3,Среднее,Состою в браке,Торговля,Рабочий,Индивидуальный предприниматель,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,12000,Кабардино-Балкария,Кабардино-Балкария,Кабардино-Балкария,Ставропольский край,ЮЖНЫЙ,1,1,1,1,0,0,0,0,0,0,0,0,21650,6,4000,0,1,137,97.000,1,0,1,1,1,6,1,1,4020,4020,nan
2,59910525,0,52,1,0,1,4,0,Неполное среднее,Состою в браке,Информационные технологии,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,9000,Иркутская область,Иркутская область,Иркутская область,Иркутская область,ВОСТОЧНО-СИБИРСКИЙ,1,1,1,1,1,0,0,0,1,0,0,0,33126,12,4000,0,1,251,84.000,0,0,1,2,1,11,0,0,0,0,nan
3,59910803,0,39,1,0,1,1,1,Высшее,Состою в браке,Образование,Руководитель среднего звена,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 20000 до 50000 руб.,25000,Ростовская область,Ростовская область,Ростовская область,Ростовская область,ЮЖНЫЙ,0,1,0,0,1,1,0,0,0,0,0,0,"8491,82",6,5000,0,0,36,168.000,1,1,1,1,1,6,3,1,"1589,923333",1590,nan
4,59911781,0,30,1,0,0,0,0,Среднее,Состою в браке,Государственная служба,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,12000,Кабардино-Балкария,Кабардино-Балкария,Кабардино-Балкария,Ставропольский край,ЮЖНЫЙ,1,1,1,1,0,0,0,0,1,0,0,0,21990,12,4000,0,1,83,101.000,1,0,1,2,1,16,2,1,"1152,15",2230,nan


In [ ]:

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15223 entries, 0 to 15222
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   AGREEMENT_RK                15223 non-null  int64  
 1   TARGET                      15223 non-null  int64  
 2   AGE                         15223 non-null  int64  
 3   SOCSTATUS_WORK_FL           15223 non-null  int64  
 4   SOCSTATUS_PENS_FL           15223 non-null  int64  
 5   GENDER                      15223 non-null  int64  
 6   CHILD_TOTAL                 15223 non-null  int64  
 7   DEPENDANTS                  15223 non-null  int64  
 8   EDUCATION                   15223 non-null  object 
 9   MARITAL_STATUS              15223 non-null  object 
 10  GEN_INDUSTRY                13856 non-null  object 
 11  GEN_TITLE                   13856 non-null  object 
 12  ORG_TP_STATE                13856 non-null  object 
 13  ORG_TP_FCAPITAL             138

In [ ]:

object_columns = data.columns[data.columns.str.contains('FL|GENDER|OWN_AUTO|AGREEMENT')]
data[object_columns] = data[object_columns].astype('object')

for col in ['PERSONAL_INCOME', 'CREDIT', 'FST_PAYMENT', 
            'LOAN_AVG_DLQ_AMT', 'LOAN_MAX_DLQ_AMT']:
    data[col] = data[col].str.replace(',', '.').astype('float')
    

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15223 entries, 0 to 15222
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   AGREEMENT_RK                15223 non-null  object 
 1   TARGET                      15223 non-null  int64  
 2   AGE                         15223 non-null  int64  
 3   SOCSTATUS_WORK_FL           15223 non-null  object 
 4   SOCSTATUS_PENS_FL           15223 non-null  object 
 5   GENDER                      15223 non-null  object 
 6   CHILD_TOTAL                 15223 non-null  int64  
 7   DEPENDANTS                  15223 non-null  int64  
 8   EDUCATION                   15223 non-null  object 
 9   MARITAL_STATUS              15223 non-null  object 
 10  GEN_INDUSTRY                13856 non-null  object 
 11  GEN_TITLE                   13856 non-null  object 
 12  ORG_TP_STATE                13856 non-null  object 
 13  ORG_TP_FCAPITAL             138

In [ ]:

work_cols = data.columns[data.columns.str.contains('GEN_I|GEN_T|JOB|ORG_')].tolist()

for col in work_cols:
    data[col] = np.where((data[col].isnull()) & (data['SOCSTATUS_PENS_FL'] == 1),
                         'Не указано', data[col])

for col in work_cols:
    data[col] = np.where((data[col].isnull()) & (data['SOCSTATUS_PENS_FL'] == 0),
                         data[col].value_counts().index[0], data[col])

data['REGION_NM'] = np.where(data['REGION_NM'].isnull(), 
                             data['REGION_NM'].value_counts().index[0], 
                             data['REGION_NM'])


data['TP_PROVINCE'] = np.where(data['TP_PROVINCE'].isnull(), 
                               data['FACT_ADDRESS_PROVINCE'], 
                               data['TP_PROVINCE'])

data['PREVIOUS_CARD_NUM_UTILIZED'] = np.where(data['PREVIOUS_CARD_NUM_UTILIZED'].isnull(), 0, 
                                              data['PREVIOUS_CARD_NUM_UTILIZED'])

In [ ]:

data.select_dtypes(include='object').describe()

,AGREEMENT_RK,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,EDUCATION,MARITAL_STATUS,GEN_INDUSTRY,GEN_TITLE,ORG_TP_STATE,ORG_TP_FCAPITAL,JOB_DIR,FAMILY_INCOME,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,TP_PROVINCE,REGION_NM,REG_FACT_FL,FACT_POST_FL,REG_POST_FL,REG_FACT_POST_FL,REG_FACT_POST_TP_FL,FL_PRESENCE_FL,OWN_AUTO,AUTO_RUS_FL,HS_PRESENCE_FL,COT_PRESENCE_FL,GAR_PRESENCE_FL,LAND_PRESENCE_FL,DL_DOCUMENT_FL,GPF_DOCUMENT_FL,FACT_PHONE_FL,REG_PHONE_FL,GEN_PHONE_FL
count,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223
unique,15223,2,2,2,7,5,32,13,6,3,11,5,81,81,80,72,11,2,2,2,2,2,2,3,2,2,2,2,2,1,2,2,2,2
top,64585727,1,0,1,Среднее специальное,Состою в браке,Торговля,Специалист,Частная компания,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,Кемеровская область,Кемеровская область,Краснодарский край,Краснодарский край,ЮЖНЫЙ,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,0,1
freq,1,13847,13176,9964,6518,9416,2386,7011,6524,13689,11453,7077,674,674,674,790,2180,12765,14480,13493,12758,13782,10507,13453,14097,11940,15074,14930,14558,15223,8961,9311,14389,13855


In [ ]:

data.drop(['AGREEMENT_RK', 'DL_DOCUMENT_FL'], axis=1, inplace=True)

In [ ]:

for col in data.select_dtypes(include='object').columns:
    print(data[col].value_counts(dropna=False))

1    13847
0     1376
Name: SOCSTATUS_WORK_FL, dtype: int64
0    13176
1     2047
Name: SOCSTATUS_PENS_FL, dtype: int64
1    9964
0    5259
Name: GENDER, dtype: int64
Среднее специальное               6518
Среднее                           4679
Высшее                            3133
Неоконченное высшее                532
Неполное среднее                   340
Два и более высших образования      20
Ученая степень                       1
Name: EDUCATION, dtype: int64
Состою в браке        9416
Не состоял в браке    3622
Разведен(а)           1245
Вдовец/Вдова           588
Гражданский брак       352
Name: MARITAL_STATUS, dtype: int64
Торговля                                     2386
Другие сферы                                 1709
Не указано                                   1366
Металлургия/Промышленность/Машиностроение    1356
Государственная служба                       1286
Здравоохранение                              1177
Образование                                   998
Транспорт 

In [ ]:

data.loc[data['REGION_NM'] == 'ПОВОЛЖСКИЙ', 'REGION_NM'] = 'ПРИВОЛЖСКИЙ'

In [ ]:

ed_lst = ['Ученая степень', 'Два и более высших образования']
data.loc[data['EDUCATION'].isin(ed_lst), 'EDUCATION'] = 'Высшее'

In [ ]:

data.loc[data['OWN_AUTO'] == '2', 'OWN_AUTO'] = '1'

In [ ]:

dct = {'Агинский Бурятский АО': 'Забайкальский край', 
       'Читинская область': 'Забайкальский край',
       'Усть-Ордынский Бурятский АО': 'Иркутская область',
       'Эвенкийский АО': 'Красноярский край'}


for col in ['REG_ADDRESS_PROVINCE', 'POSTAL_ADDRESS_PROVINCE', 
            'FACT_ADDRESS_PROVINCE', 'TP_PROVINCE']:
    data[col].replace(dct, inplace=True)

In [ ]:

for col in ['REG_ADDRESS_PROVINCE', 'POSTAL_ADDRESS_PROVINCE', 'FACT_ADDRESS_PROVINCE']:
    data[col] = np.where((data[col] == 'Хакасия')
                       | (data[col] == 'Ямало-Ненецкий АО')
                       | (data[col] == 'Магаданская область')
                       | (data[col] == 'Калмыкия')
                       | (data[col] == 'Дагестан')
                       | (data[col] == 'Чечня'), 
                       'ДРУГОЕ', data[col])


data['TP_PROVINCE'] = np.where((data['TP_PROVINCE'] == 'Сахалинская область')
                               | (data['TP_PROVINCE'] == 'Еврейская АО')
                               | (data['TP_PROVINCE'] == 'Магаданская область')
                               | (data['TP_PROVINCE'] == 'Дагестан')
                               | (data['TP_PROVINCE'] == 'Кабардино-Балкария'), 
                               'ДРУГОЕ', data['TP_PROVINCE'])

In [ ]:

jobdir_lst = ['Реклама и маркетинг', 'Кадровая служба и секретариат',
              'Пр-техн. обесп. и телеком.', 'Юридическая служба']
data.loc[data['JOB_DIR'].isin(jobdir_lst), 'JOB_DIR'] = 'Другое'

In [ ]:

pd.set_option('display.float_format', lambda x: '%.3f' % x)
data.describe()

,TARGET,AGE,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,CREDIT,TERM,FST_PAYMENT,FACT_LIVING_TERM,WORK_TIME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_NUM_PAYM,LOAN_DLQ_NUM,LOAN_MAX_DLQ,LOAN_AVG_DLQ_AMT,LOAN_MAX_DLQ_AMT,PREVIOUS_CARD_NUM_UTILIZED
count,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,13855.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000
mean,0.119,40.406,1.099,0.645,13853.836,14667.959,8.101,3398.563,3039.340,292.212,1.388,0.752,7.876,0.195,0.130,239.457,243.022,0.019
std,0.324,11.601,0.995,0.812,9015.468,12147.873,4.094,5158.109,262455.437,24364.832,0.794,0.988,7.956,0.685,0.356,827.700,836.100,0.138
min,0.000,21.000,0.000,0.000,24.000,2000.000,3.000,0.000,-26.000,1.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000
25%,0.000,30.000,0.000,0.000,8000.000,6500.000,6.000,1000.000,41.000,24.000,1.000,0.000,4.000,0.000,0.000,0.000,0.000,0.000
50%,0.000,39.000,1.000,0.000,12000.000,11550.000,6.000,2000.000,108.000,48.000,1.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000
75%,0.000,50.000,2.000,1.000,17000.000,19170.000,10.000,4000.000,204.000,110.000,2.000,1.000,8.000,0.000,0.000,0.000,0.000,0.000
max,1.000,67.000,10.000,7.000,250000.000,119700.000,36.000,140000.000,28101997.000,2867959.000,11.000,11.000,114.000,13.000,8.000,28012.200,28012.200,2.000


In [ ]:

data['FACT_LIVING_TERM'] = data['FACT_LIVING_TERM'].abs()

In [ ]:

data['FACT_TP_FL'] = np.where(data['FACT_ADDRESS_PROVINCE'] == data['TP_PROVINCE'], 
                              1, 0).astype('object')

In [ ]:

a = 0.001

data['PAYMENT'] = (data['CREDIT'] + a) / (data['TERM'] + a)
data['SHARE_INCOME'] = (data['PERSONAL_INCOME'] + a) / (data['CREDIT'] + a)
data['LOAN_NUM_OPEN'] = data['LOAN_NUM_TOTAL'] - data['LOAN_NUM_CLOSED']
data['PAYM_TO_LOAN'] = (data['LOAN_NUM_PAYM'] + a) / (data['LOAN_NUM_TOTAL'] + a)
data['DLQ_TO_PAYM'] = (data['LOAN_DLQ_NUM'] + a) / (data['LOAN_NUM_PAYM'] + a)
data['FST_SHARE'] = (data['FST_PAYMENT'] + a) / (data['FST_PAYMENT'] + data['CREDIT'])

In [ ]:

def collapse_categories(X):
    dct = {'ЮЖНЫЙ': 'cat1', 
           'ЦЕНТРАЛЬНЫЙ 1': 'cat1', 
           'ЦЕНТРАЛЬНЫЙ 2': 'cat1',
           'ЗАПАДНО-СИБИРСКИЙ': 'cat2', 
           'ВОСТОЧНО-СИБИРСКИЙ': 'cat2', 
           'УРАЛЬСКИЙ': 'cat2',
           'ДАЛЬНЕВОСТОЧНЫЙ': 'cat2', 
           'СЕВЕРО-ЗАПАДНЫЙ': 'cat2', 
           'ЦЕНТРАЛЬНЫЙ ОФИС': 'cat2',
           'ПРИВОЛЖСКИЙ': 'cat3'}
    X.replace(dct, inplace=True)
    return X

In [ ]:

bins = [-np.inf, 5000, 10000, 20000, 50000, np.inf]
lab = ['до 5000 руб.', 'от 5000 до 10000 руб.', 'от 10000 до 20000 руб.', 
       'от 20000 до 50000 руб.', 'свыше 50000 руб.']
data['PERSONAL_INCOME_CAT'] = pd.cut(data['PERSONAL_INCOME'], bins, labels=lab)

In [ ]:

data['PERSONAL_FAMILY_INCOME'] = np.where(data['PERSONAL_INCOME_CAT'] == data['FAMILY_INCOME'], 
                                          1, 0).astype('object')

# удаляем переменную PERSONAL_INCOME_CAT
data.drop('PERSONAL_INCOME_CAT', axis=1, inplace=True)

In [ ]:

data['FACT_LIVING_TERM'] = np.where(data['FACT_LIVING_TERM'] / 12 > data['AGE'],
                                     np.NaN, data['FACT_LIVING_TERM'])

In [ ]:

data['WORK_TIME'] = np.where((data['AGE'] - data['WORK_TIME'] / 12) < 16, 
                              np.NaN, data['WORK_TIME'])

In [ ]:

train, test, y_train, y_test = train_test_split(data.drop('TARGET', axis=1), 
                                                data['TARGET'], 
                                                test_size=.3, 
                                                stratify=data['TARGET'], 
                                                random_state=100)

In [ ]:

train['GEN_TITLE'].value_counts(dropna=False)

Специалист                        4886
Рабочий                           2175
Не указано                         967
Служащий                           623
Руководитель среднего звена        484
Работник сферы услуг               410
Высококвалифиц. специалист         382
Руководитель высшего звена         300
Индивидуальный предприниматель     152
Другое                             108
Руководитель низшего звена          98
Военнослужащий по контракту         62
Партнер                              9
Name: GEN_TITLE, dtype: int64

In [ ]:
# создаем собственный класс RareGrouper, укрупняющий редкие категории
class RareGrouper(BaseEstimator, TransformerMixin): 
    """
        Параметры:
        - threshold: минимально допустимая абсолютная частота, при которой замены не происходит
    """
    
    def __init__(self, threshold=20):

        self.d = defaultdict(list)
        self.threshold = threshold
        
    def fit(self, X, y=None):
        for col in X.columns:
            abs_freq = X[col].value_counts()
            self.d[col] = abs_freq[abs_freq >= self.threshold].index.tolist()
        return self

    def transform(self, X):
        X = X.copy()
        for col in X.columns:
            X[col] = np.where(X[col].isin(self.d[col]), X[col], 'Other')
        return X

In [ ]:

train_copy = train.copy()
raregrouper = RareGrouper(threshold=70)
raregrouper.fit(train_copy[['GEN_TITLE']])
train_copy['GEN_TITLE'] = raregrouper.transform(train_copy[['GEN_TITLE']])
train_copy['GEN_TITLE'].value_counts(dropna=False)

Специалист                        4886
Рабочий                           2175
Не указано                         967
Служащий                           623
Руководитель среднего звена        484
Работник сферы услуг               410
Высококвалифиц. специалист         382
Руководитель высшего звена         300
Индивидуальный предприниматель     152
Другое                             108
Руководитель низшего звена          98
Other                               71
Name: GEN_TITLE, dtype: int64

In [ ]:
# создаем собственный класс RareGrouper2, укрупняющий редкие категории
class RareGrouper2(TransformerMixin, BaseEstimator):
    def __init__(self,  min_freq=20, max_features=None, include_na=True, features=None, 
                 fill_str='other', fill_num=-1, return_df=False):
        
        self.features = features
        self.max_features = max_features
        self.min_freq = min_freq
        self.include_na = include_na
        self.fill_str = fill_str
        self.fill_num = fill_num
        self.return_df = return_df
             
    def fit(self, X, y=None, **fit_params):
        assert len(X.shape) == 2, 'Array must be 2-dim'
        
        self.fmap = None
        if 'fmap' in fit_params:
            self.fmap = fit_params['fmap']
   
        X = DataFrame(X, columns=self.fmap)

        if self.features is None:
            features = X.columns
        else:
            features = self.features
        
        self.feats_dict = {}
        
        for i in features:
            cnt = Counter(X[i])
            if self.include_na:
                keeplist = set([x for x, f in cnt.most_common(self.max_features) if f >= self.min_freq])

            else:
                keeplist = set([x for x, f in cnt.most_common(self.max_features) if f >= self.min_freq \
                                and x is not np.nan])
            if np.issubdtype(X[i].dtype, np.number):
                self.feats_dict[i] = (keeplist, self.fill_num)
            else:
                self.feats_dict[i] = (keeplist, self.fill_str)
        return self
    
    def transform(self, X, y=None):        
        assert len(X.shape) == 2, 'Array must be 2-dim'
        assert 'feats_dict' in self.__dict__, 'Model should be fitted first'
        
        out = DataFrame(X, columns=self.fmap)
        for i in self.feats_dict:
            l, v = self.feats_dict[i]
            out.loc[~out[i].isin(l), i] = v
        if self.return_df:
            return out
        else:
            return out.values

In [ ]:

class Replacer(BaseEstimator, TransformerMixin):
    """
    Параметры:
        repl: значение для замены
    """
    def __init__(self, repl_value=0.1):
        self.repl_value = repl_value
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_replaced = np.where(X <= 0, self.repl_value, X)
        return X_replaced

In [ ]:

class CorrLog(BaseEstimator, TransformerMixin):
    """
    Параметры:
    lower: нижнее пороговое значение
    k: корректировка асимметрии
    """
    def __init__(self, lower=0.001, k=0.2):

        self.lower = lower
        self.k = k
        self.mean = None
                
    def fit(self, X, y=None):
        self.mean = np.mean(X)
        return self
    
    def transform(self, X):
        X = np.log(np.clip(X, self.lower, None) / self.mean + self.k)
        return X

In [ ]:

cat_columns = train.select_dtypes(include='object').columns

num_columns = train.select_dtypes(exclude='object').columns
regionnm_regroup = ['REGION_NM']
genindustry_delrare = ['GEN_INDUSTRY']
gentitle_delrare = ['GEN_TITLE']
cat_columns = list(set(cat_columns).difference(set(
    regionnm_regroup + genindustry_delrare + gentitle_delrare)))
cat_columns

['REG_ADDRESS_PROVINCE',
 'FACT_PHONE_FL',
 'JOB_DIR',
 'GAR_PRESENCE_FL',
 'FL_PRESENCE_FL',
 'MARITAL_STATUS',
 'FACT_ADDRESS_PROVINCE',
 'LAND_PRESENCE_FL',
 'SOCSTATUS_PENS_FL',
 'PERSONAL_FAMILY_INCOME',
 'EDUCATION',
 'GENDER',
 'FACT_POST_FL',
 'AUTO_RUS_FL',
 'REG_FACT_FL',
 'SOCSTATUS_WORK_FL',
 'FAMILY_INCOME',
 'OWN_AUTO',
 'GEN_PHONE_FL',
 'TP_PROVINCE',
 'REG_PHONE_FL',
 'GPF_DOCUMENT_FL',
 'REG_POST_FL',
 'COT_PRESENCE_FL',
 'REG_FACT_POST_FL',
 'POSTAL_ADDRESS_PROVINCE',
 'ORG_TP_FCAPITAL',
 'FACT_TP_FL',
 'HS_PRESENCE_FL',
 'REG_FACT_POST_TP_FL',
 'ORG_TP_STATE']

In [ ]:

num_columns_log = ['PERSONAL_INCOME', 'PAYMENT', 'CREDIT', 'LOAN_AVG_DLQ_AMT']
worktime_corrlog0_01 = ['WORK_TIME']
fctlivingterm_corrlog0_5 = ['FACT_LIVING_TERM']
fstpayment_corrlog0_1 = ['FST_PAYMENT']

comb_log_lst = (num_columns_log + worktime_corrlog0_01 + 
                fctlivingterm_corrlog0_5 + fstpayment_corrlog0_1)

num_columns = list(set(num_columns).difference(set(comb_log_lst)))

num_columns

['LOAN_NUM_CLOSED',
 'LOAN_MAX_DLQ',
 'LOAN_MAX_DLQ_AMT',
 'DLQ_TO_PAYM',
 'SHARE_INCOME',
 'LOAN_NUM_OPEN',
 'FST_SHARE',
 'PREVIOUS_CARD_NUM_UTILIZED',
 'LOAN_DLQ_NUM',
 'DEPENDANTS',
 'LOAN_NUM_PAYM',
 'LOAN_NUM_TOTAL',
 'TERM',
 'PAYM_TO_LOAN',
 'CHILD_TOTAL',
 'AGE']

In [ ]:

num_pipe = Pipeline([
    ('scaler', StandardScaler())
])

num_log_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='median')),
    ('repl', Replacer(repl_value=0.1)),
    ('log', FunctionTransformer(np.log, validate=False)),
    ('scaler', StandardScaler())
])

worktime_corrlog0_01_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='median')),
    ('corrlog0_01', CorrLog(k=0.01)),
    ('scaler', StandardScaler())
])

fctlivingterm_corrlog0_5_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='median')),
    ('corrlog0_5', CorrLog(k=0.5)),
    ('scaler', StandardScaler())
])

fstpayment_corrlog0_1_pipe = Pipeline([
    ('corrlog0_1', CorrLog(k=0.1)),
    ('scaler', StandardScaler())
])

regionnm_regroup_pipe = Pipeline([
    ('regroup', FunctionTransformer(collapse_categories, validate=False)),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

gentitle_delrare_pipe = Pipeline([
    ('raregroup', RareGrouper()),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

genindustry_delrare_pipe = Pipeline([
    ('raregroup2', RareGrouper2()),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

cat_pipe = Pipeline([
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [ ]:

transformers = [('num', num_pipe, num_columns),
                ('num_log', num_log_pipe, num_columns_log),
                ('worktime_corrlog0_01', worktime_corrlog0_01_pipe, worktime_corrlog0_01),
                ('fctlivingterm_corrlog0_5', fctlivingterm_corrlog0_5_pipe, fctlivingterm_corrlog0_5),
                ('fstpayment_corrlog0_1', fstpayment_corrlog0_1_pipe, fstpayment_corrlog0_1),
                ('regionnm_regroup', regionnm_regroup_pipe, regionnm_regroup),
                ('gentitle_delrare', gentitle_delrare_pipe, gentitle_delrare),
                ('genindustry_delrare', genindustry_delrare_pipe, genindustry_delrare),
                ('cat', cat_pipe, cat_columns)]

transformer = ColumnTransformer(transformers=transformers)

In [ ]:

pipe = Pipeline([('tf', transformer), 
                 ('logreg', LogisticRegression(solver='liblinear', 
                                               penalty='l1', 
                                               random_state=42))])

In [ ]:

param_grid = {'tf__gentitle_delrare__raregroup__threshold': [20, 300],
              'tf__genindustry_delrare__raregroup2__min_freq': [300, 500],
              'logreg__C': np.logspace(-1, -0.5, 5)}

In [ ]:

strat = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:

gs = GridSearchCV(pipe, 
                  param_grid, 
                  scoring='roc_auc',
                  cv=strat, 
                  return_train_score=False)

In [ ]:

gs.fit(train, y_train)

print('Наилучшие значения гиперпараметров: {}'.format(gs.best_params_))
print('Наилучшее значение AUC: {:.3f}'.format(gs.best_score_))
print('AUC на тестовом наборе: {:.3f}'.format(
    roc_auc_score(y_test, gs.predict_proba(test)[:, 1])))

Наилучшие значения гиперпараметров: {'logreg__C': 0.1, 'tf__genindustry_delrare__raregroup2__min_freq': 300, 'tf__gentitle_delrare__raregroup__threshold': 20}
Наилучшее значение AUC: 0.704
AUC на тестовом наборе: 0.691


In [ ]:

fulldata = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Артем Груздев Курс/Course_ML/Модуль_3/Data/Credit_OTP.csv', encoding='cp1251', sep=';')
fulldata.head()


,AGREEMENT_RK,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,EDUCATION,MARITAL_STATUS,GEN_INDUSTRY,GEN_TITLE,ORG_TP_STATE,ORG_TP_FCAPITAL,JOB_DIR,FAMILY_INCOME,PERSONAL_INCOME,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,TP_PROVINCE,REGION_NM,REG_FACT_FL,FACT_POST_FL,REG_POST_FL,REG_FACT_POST_FL,REG_FACT_POST_TP_FL,FL_PRESENCE_FL,OWN_AUTO,AUTO_RUS_FL,HS_PRESENCE_FL,COT_PRESENCE_FL,GAR_PRESENCE_FL,LAND_PRESENCE_FL,CREDIT,TERM,FST_PAYMENT,DL_DOCUMENT_FL,GPF_DOCUMENT_FL,FACT_LIVING_TERM,WORK_TIME,FACT_PHONE_FL,REG_PHONE_FL,GEN_PHONE_FL,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_NUM_PAYM,LOAN_DLQ_NUM,LOAN_MAX_DLQ,LOAN_AVG_DLQ_AMT,LOAN_MAX_DLQ_AMT,PREVIOUS_CARD_NUM_UTILIZED
0,59910150,0,49,1,0,1,2,1,Среднее специальное,Состою в браке,Торговля,Рабочий,Частная компания,Без участия,Вспомогательный техперсонал,от 10000 до 20000 руб.,5000,Оренбургская область,Оренбургская область,Оренбургская область,Оренбургская область,ПОВОЛЖСКИЙ,1,1,1,1,1,0,0,0,0,0,0,0,8000,6,8650,0,1,220,18.000,0,0,1,1,1,6,2,1,1580,1580,nan
1,59910230,0,32,1,0,1,3,3,Среднее,Состою в браке,Торговля,Рабочий,Индивидуальный предприниматель,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,12000,Кабардино-Балкария,Кабардино-Балкария,Кабардино-Балкария,Ставропольский край,ЮЖНЫЙ,1,1,1,1,0,0,0,0,0,0,0,0,21650,6,4000,0,1,137,97.000,1,0,1,1,1,6,1,1,4020,4020,nan
2,59910525,0,52,1,0,1,4,0,Неполное среднее,Состою в браке,Информационные технологии,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,9000,Иркутская область,Иркутская область,Иркутская область,Иркутская область,ВОСТОЧНО-СИБИРСКИЙ,1,1,1,1,1,0,0,0,1,0,0,0,33126,12,4000,0,1,251,84.000,0,0,1,2,1,11,0,0,0,0,nan
3,59910803,0,39,1,0,1,1,1,Высшее,Состою в браке,Образование,Руководитель среднего звена,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 20000 до 50000 руб.,25000,Ростовская область,Ростовская область,Ростовская область,Ростовская область,ЮЖНЫЙ,0,1,0,0,1,1,0,0,0,0,0,0,"8491,82",6,5000,0,0,36,168.000,1,1,1,1,1,6,3,1,"1589,923333",1590,nan
4,59911781,0,30,1,0,0,0,0,Среднее,Состою в браке,Государственная служба,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,12000,Кабардино-Балкария,Кабардино-Балкария,Кабардино-Балкария,Ставропольский край,ЮЖНЫЙ,1,1,1,1,0,0,0,0,1,0,0,0,21990,12,4000,0,1,83,101.000,1,0,1,2,1,16,2,1,"1152,15",2230,nan


In [ ]:

custprep.preprocessing(fulldata)
fulldata.head()

,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,EDUCATION,MARITAL_STATUS,GEN_INDUSTRY,GEN_TITLE,ORG_TP_STATE,ORG_TP_FCAPITAL,JOB_DIR,FAMILY_INCOME,PERSONAL_INCOME,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,TP_PROVINCE,REGION_NM,REG_FACT_FL,FACT_POST_FL,REG_POST_FL,REG_FACT_POST_FL,REG_FACT_POST_TP_FL,FL_PRESENCE_FL,OWN_AUTO,AUTO_RUS_FL,HS_PRESENCE_FL,COT_PRESENCE_FL,GAR_PRESENCE_FL,LAND_PRESENCE_FL,CREDIT,TERM,FST_PAYMENT,GPF_DOCUMENT_FL,FACT_LIVING_TERM,WORK_TIME,FACT_PHONE_FL,REG_PHONE_FL,GEN_PHONE_FL,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_NUM_PAYM,LOAN_DLQ_NUM,LOAN_MAX_DLQ,LOAN_AVG_DLQ_AMT,LOAN_MAX_DLQ_AMT,PREVIOUS_CARD_NUM_UTILIZED,FACT_TP_FL,PAYMENT,SHARE_INCOME,LOAN_NUM_OPEN,PAYM_TO_LOAN,DLQ_TO_PAYM,FST_SHARE,PERSONAL_FAMILY_INCOME
0,0,49,1,0,1,2,1,Среднее специальное,Состою в браке,Торговля,Рабочий,Частная компания,Без участия,Вспомогательный техперсонал,от 10000 до 20000 руб.,5000.000,Оренбургская область,Оренбургская область,Оренбургская область,Оренбургская область,ПРИВОЛЖСКИЙ,1,1,1,1,1,0,0,0,0,0,0,0,8000.000,6,8650.000,1,220.000,18.000,0,0,1,1,1,6,2,1,1580.000,1580.000,0.000,1,1333.111,0.625,0,5.995,0.333,0.520,0
1,0,32,1,0,1,3,3,Среднее,Состою в браке,Торговля,Рабочий,Индивидуальный предприниматель,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,12000.000,Кабардино-Балкария,Кабардино-Балкария,Кабардино-Балкария,Ставропольский край,ЮЖНЫЙ,1,1,1,1,0,0,0,0,0,0,0,0,21650.000,6,4000.000,1,137.000,97.000,1,0,1,1,1,6,1,1,4020.000,4020.000,0.000,0,3607.732,0.554,0,5.995,0.167,0.156,1
2,0,52,1,0,1,4,0,Неполное среднее,Состою в браке,Информационные технологии,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,9000.000,Иркутская область,Иркутская область,Иркутская область,Иркутская область,ВОСТОЧНО-СИБИРСКИЙ,1,1,1,1,1,0,0,0,1,0,0,0,33126.000,12,4000.000,1,251.000,84.000,0,0,1,2,1,11,0,0,0.000,0.000,0.000,1,2760.270,0.272,1,5.498,0.000,0.108,0
3,0,39,1,0,1,1,1,Высшее,Состою в браке,Образование,Руководитель среднего звена,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 20000 до 50000 руб.,25000.000,Ростовская область,Ростовская область,Ростовская область,Ростовская область,ЮЖНЫЙ,0,1,0,0,1,1,0,0,0,0,0,0,8491.820,6,5000.000,0,36.000,168.000,1,1,1,1,1,6,3,1,1589.923,1590.000,0.000,1,1415.068,2.944,0,5.995,0.500,0.371,1
4,0,30,1,0,0,0,0,Среднее,Состою в браке,Государственная служба,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,12000.000,Кабардино-Балкария,Кабардино-Балкария,Кабардино-Балкария,Ставропольский край,ЮЖНЫЙ,1,1,1,1,0,0,0,0,1,0,0,0,21990.000,12,4000.000,1,83.000,101.000,1,0,1,2,1,16,2,1,1152.150,2230.000,0.000,0,1832.347,0.546,1,7.997,0.125,0.154,1


In [ ]:

y_fulldata = fulldata.pop('TARGET')

In [ ]:

best_params = gs.best_params_
pipe.set_params(**best_params)
pipe.fit(fulldata, y_fulldata)
print('AUC на всей исторической выборке: {:.3f}'.format(
    roc_auc_score(y_fulldata, pipe.predict_proba(fulldata)[:, 1])))

AUC на всей исторической выборке: 0.718


In [ ]:

newdata = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Артем Груздев Курс/Course_ML/Модуль_3/Data/Credit_OTP_new.csv', encoding='cp1251', sep=';')

In [ ]:

custprep.preprocessing(newdata)
y_new = newdata.pop('TARGET').values

In [ ]:

print("AUC на выборке новых данных: {:.3f}".
      format(roc_auc_score(y_new, pipe.predict_proba(newdata)[:, 1])))

AUC на выборке новых данных: 0.707
